# Patent Kaggle challenge

notebook from fastai course lesson 4

## Importing kaggle dataset

In [3]:
!kaggle competitions download -c us-patent-phrase-to-phrase-matching
!unzip us-patent-phrase-to-phrase-matching.zip -d us-patent-phrase-to-phrase-matching
!rm us-patent-phrase-to-phrase-matching.zip

In [1]:
from pathlib import Path

path = Path('us-patent-phrase-to-phrase-matching')
!ls {path}

sample_submission.csv  test.csv  train.csv


## Viewing data

In [2]:
import pandas as pd
df = pd.read_csv(path/'train.csv')
df.head()

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


In [3]:
df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,8d135da0b55b8c88,component composite coating,composition,H01
freq,1,152,24,2186


In [4]:
df['input'] = 'TEXT1: ' + df.anchor + '; TEXT2: ' + df.target + '; CONTEXT: ' + df.context
df.input.head()

0    TEXT1: abatement; TEXT2: abatement of pollutio...
1    TEXT1: abatement; TEXT2: act of abating; CONTE...
2    TEXT1: abatement; TEXT2: active catalyst; CONT...
3    TEXT1: abatement; TEXT2: eliminating process; ...
4    TEXT1: abatement; TEXT2: forest region; CONTEX...
Name: input, dtype: object

## Tokenisation

In [49]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(df)
ds = ds.rename_columns({'score':'labels'})
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'labels', 'input'],
    num_rows: 36473
})

In [17]:
model_nm = 'microsoft/deberta-v3-small'
model_nm_variant = 'bert-base-cased'

In [18]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm, use_fast = False)
tokz_variant = AutoTokenizer.from_pretrained(model_nm_variant)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [19]:
tokz.tokenize("G'day folks, I'm Jeremy from fast.ai!")

['▁G',
 "'",
 'day',
 '▁folks',
 ',',
 '▁I',
 "'",
 'm',
 '▁Jeremy',
 '▁from',
 '▁fast',
 '.',
 'ai',
 '!']

In [22]:
tokz_variant.tokenize("G'day folks, I'm Jeremy from fast.ai!")

['G',
 "'",
 'day',
 'folks',
 ',',
 'I',
 "'",
 'm',
 'Jeremy',
 'from',
 'fast',
 '.',
 'a',
 '##i',
 '!']

In [20]:
tokz.tokenize("A platypus is an ornithorhynchus anatinus.")

['▁A',
 '▁platypus',
 '▁is',
 '▁an',
 '▁or',
 'ni',
 'tho',
 'rhynch',
 'us',
 '▁an',
 'at',
 'inus',
 '.']

In [21]:
tokz_variant.tokenize("A platypus is an ornithorhynchus anatinus.")

['A',
 'p',
 '##lat',
 '##y',
 '##pus',
 'is',
 'an',
 'or',
 '##ni',
 '##th',
 '##or',
 '##hy',
 '##nch',
 '##us',
 'an',
 '##ati',
 '##nus',
 '.']

In [50]:
def tok_func(x): return tokz(x["input"])
tok_ds = ds.map(tok_func, batched=True)
tok_ds

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 36473
})

In [51]:
row = tok_ds[0]
row['input'], row['input_ids']

('TEXT1: abatement; TEXT2: abatement of pollution; CONTEXT: A47',
 [1,
  54453,
  435,
  294,
  47284,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  20967,
  104917,
  294,
  336,
  5753,
  2])

In [52]:
print(len(tokz.vocab))
print(tokz.vocab['▁of'])

128000
265


In [53]:
print(ord('▁'))
print(ord('_'))

9601
95


Test Set 

In [54]:
eval_df = pd.read_csv(path/'test.csv')
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,hybrid bearing,inorganic photoconductor drum,G02
freq,1,2,1,3
